In [1]:
import numpy as np
import os
import sys
from PIL import Image
from matplotlib.pyplot import figure
import pandas
import matplotlib.pyplot as plt
import matplotlib.ticker as tick
import torch
from torch import nn
import torch.optim as optim 
from torch.utils.data import TensorDataset, DataLoader

class MLP(nn.Module):
  '''
    Multilayer Perceptron.
  '''
  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
      nn.Linear(256, 128),
      nn.ReLU(),
      nn.Linear(128, 128),
      nn.ReLU(),
      nn.Linear(128, 256),
    )


  def forward(self, x):
    '''Forward pass'''
    return self.layers(x)

def predict(inputVector, weightMatrix):
        '''
        Predict the output of the input vector.
        '''
        
        predictionVector = []
        for i in range(len(weightMatrix)):
            row_sum = sum(
                (inputVector[j] * weightMatrix[j][i])
                for j in range(len(weightMatrix[0]) - 1)
            )

            predictionVector.append(self.activation_fn(row_sum))
           
        return predictionVector

def add_noise(outputVector,noise_percent,stdev):
    '''
    Add noise to the output vector.
    '''
    mean = 0
    noise = np.random.normal(mean,stdev,outputVector.shape) * noise_percent
    return outputVector + noise

def Noisy_Testing(stdev, testRounds, inputImageVectors):
    '''
    Test the DNN with noise.
    '''
    tableObject = {}
    plotObject = {'fh': [], 'ffa': []}

    for i in range(len(stdev)):
        tableObject['std_'+ str(stdev[i]) + '_fh'] = []
        tableObject['std_'+ str(stdev[i]) + '_ffa'] = []

    for j in range(len(stdev)):   
        for k in range(testRounds) :
            corruptedVector = add_noise(inputImageVectors[k],0.1,stdev[j]) 
            testPrediction = model(torch.from_numpy(corruptedVector.astype('float32'))).detach().numpy()
            for l in range(256):
                if output[l] > 0:
                    testPrediction[l] = 1
                else:
                    testPrediction[l] = 0
            fh,ffa = calculate_performance_metrics(inputImageVectors[k],testPrediction)
            print("FH:",fh)
            print("FFA:",ffa)

ImageVectors = []

for i in range(10):
    path = os.path.join(os.path.dirname(os.path.abspath(sys.argv[1])),'characters1', str(i) +'.png')
    im = Image.open(path, 'r')
    gray = im.convert('L')
    bw = gray.point(lambda x: 0 if x<135 else 1, '1')
    ImageVectors.append(np.array(list(bw.getdata())))
    
for i in range(26):
    x = i + 65
    path = os.path.join(os.path.dirname(os.path.abspath(sys.argv[1])),'characters1', chr(i + 65) +'.png')
    im = Image.open(path, 'r')
    gray = im.convert('L')
    bw = gray.point(lambda x: 0 if x<135 else 1, '1')
    ImageVectors.append(np.array(list(bw.getdata())))

set1 = np.array(ImageVectors)

ImageVectors = []

for i in range(10):
    path = os.path.join(os.path.dirname(os.path.abspath(sys.argv[1])),'characters2', str(i) +'.png')
    im = Image.open(path, 'r')
    gray = im.convert('L')
    bw = gray.point(lambda x: 0 if x<135 else 1, '1')
    ImageVectors.append(np.array(list(bw.getdata())))
    
for i in range(26):
    x = i + 65
    path = os.path.join(os.path.dirname(os.path.abspath(sys.argv[1])),'characters2', chr(i + 65) +'.png')
    im = Image.open(path, 'r')
    gray = im.convert('L')
    bw = gray.point(lambda x: 0 if x<135 else 1, '1')
    ImageVectors.append(np.array(list(bw.getdata())))
    
set2 = np.array(ImageVectors)

imageTensor = torch.Tensor(set1)
_dataSet = TensorDataset(imageTensor, imageTensor)
_dataLoader = DataLoader(_dataSet)

model = MLP()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(10):
    losses = []

    for batch_idx, (data, targets) in enumerate(_dataLoader):

        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        losses.append(loss.item())

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()
    
def calculate_performance_metrics(inputVector, outputVector):
    totalBlackPixelCount = sum(x == 0 for x in inputVector)
    totalWhitePixelCount = sum(x == 1 for x in inputVector)
    wrongBlackPixelCount = 0
    rightBlackPixelCount = 0
    for i in range(256):
        if outputVector[i] < 0.0001:
            if  abs(outputVector[i] - inputVector[i]) < 0.0001:
                rightBlackPixelCount += 1
            else:
                wrongBlackPixelCount += 1
    fh = rightBlackPixelCount/totalBlackPixelCount
    ffa = wrongBlackPixelCount/totalWhitePixelCount
    return fh, ffa 

model.eval()
print("Checking accuracy on Training Set")
for i in range(36):
    output = model(torch.from_numpy(set1[i].astype('float32'))).detach().numpy()
    for j in range(256):
        if output[j] > 0:
            output[j] = 1
        else:
            output[j] = 0
    fh, ffa = calculate_performance_metrics(set1[i], output)
    print("FH:", fh, "FFA:", ffa)

print("Checking accuracy on Test Set")
for i in range(36):
    output = model(torch.from_numpy(set2[i].astype('float32'))).detach().numpy()
    for j in range(256):
        if output[j] > 0:
            output[j] = 1
        else:
            output[j] = 0
    fh, ffa = calculate_performance_metrics(set2[i], output)
    print("FH:", fh, "FFA:", ffa)
print("NOISY")
stdev = [0,0.001, 0.002, 0.003, 0.005, 0.01, 0.02, 0.03, 0.05,0.1]
Noisy_Testing(stdev, 10, set1)

Checking accuracy on Training Set
FH: 1.0 FFA: 0.11724137931034483
FH: 0.9195402298850575 FFA: 0.27218934911242604
FH: 0.8994413407821229 FFA: 0.0
FH: 0.8830409356725146 FFA: 0.011764705882352941
FH: 0.8924050632911392 FFA: 0.04081632653061224
FH: 0.8809523809523809 FFA: 0.022727272727272728
FH: 0.884393063583815 FFA: 0.012048192771084338
FH: 0.9781021897810219 FFA: 0.03361344537815126
FH: 0.8608247422680413 FFA: 0.04838709677419355
FH: 0.8663101604278075 FFA: 0.0
FH: 0.9664429530201343 FFA: 0.037383177570093455
FH: 0.8835978835978836 FFA: 0.014925373134328358
FH: 0.9171974522292994 FFA: 0.010101010101010102
FH: 0.9210526315789473 FFA: 0.0
FH: 0.8256880733944955 FFA: 0.0
FH: 0.8534031413612565 FFA: 0.0
FH: 0.8042328042328042 FFA: 0.014925373134328358
FH: 0.9293478260869565 FFA: 0.0
FH: 0.9818181818181818 FFA: 0.07534246575342465
FH: 0.9375 FFA: 0.109375
FH: 0.9553072625698324 FFA: 0.03896103896103896
FH: 0.95625 FFA: 0.020833333333333332
FH: 0.9631336405529954 FFA: 0.0
FH: 0.8492462311